In [2]:
import os
import json
import torchvision.datasets
import torch
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import pandas as pd
import numpy as np
from collections import defaultdict
import glob
import json
import seaborn as sns
# comparing regular vs. dp-sgd vs. dp-sgd-auto-clip vs dp-sgd-auto-clip-psac on MNIST, Fashion-MNIST and cifar 10

def read_certified_robustness_results(key_list, test_result_df_dict, end_radius=2.5, step_size=0.1):
    sigma_results = defaultdict(lambda: defaultdict(dict))
    for key in key_list:
        test_result_df = test_result_df_dict[key]
        for sigma in [0.25, 0.5, 1.0]:
            
            # print(test_result_df)
            if f'certified_radius_{sigma}' not in test_result_df:
                continue
            cert_radius = test_result_df[f'certified_radius_{sigma}']
            correct = test_result_df[f'certified_correct_{sigma}']
            pred = test_result_df[f'certified_pred_{sigma}']
            # different_radius = []
            for radius in np.arange(0.0, end_radius, step_size):
                num_correct_at_radius = ((correct == True) & (cert_radius >= radius)).mean()
                sigma_results[sigma][key][radius] = {'certified_acc': round(100 * num_correct_at_radius, 1)}
    return sigma_results

def get_acc_df(dir_list, filter_num_augmentation=2):
    test_result_df_dict = {}
    metric_dict = defaultdict(dict)
    key_list = []
    noise_std_list = []
    for j, dir in enumerate(dir_list):
        # print(dir)
        with open(os.path.join(dir, "config.json"), "r") as f:
            cfg = json.load(f)
        
        method = cfg['method']
        if cfg['method'] in ["dpsgd-auto-clip-adv-smooth", "dpsgd-augment-auto-clip"]:
            method += f"_psac_{cfg['psac']}"
        augment_noise_std = cfg["augment_noise_std"]
        num_augmentations = cfg['num_augmentations']
        if num_augmentations != filter_num_augmentation:
            continue

        cur_key = f"{cfg['dataset']}_{method}_noise_std_{augment_noise_std}_num_aug_{num_augmentations}"
        key_list.append(cur_key)
        noise_std_list.append(augment_noise_std)

        test_result_df = pd.read_json(os.path.join(dir, "test_results.json"))
        test_result_df_dict[cur_key] = test_result_df
        num_correct_at_radius = len(test_result_df[test_result_df['correct'] == True])
        accuracy = num_correct_at_radius / len(test_result_df)

        metric_dict["dataset"][j] = cfg['dataset']
        metric_dict["method"][j] = method
        metric_dict["noise_std"][j] = augment_noise_std
        metric_dict["num_augmentation"][j] = num_augmentations
        metric_dict["accuracy"][j] = accuracy * 100
        
        # for sigma in [0.25, 0.5, 1.0]:
        sigma = augment_noise_std
        radius = test_result_df[f'certified_radius_{sigma}']
        correct = test_result_df[f'certified_correct_{sigma}']
        pred = test_result_df[f'certified_pred_{sigma}']
        certified_accuracy = len(test_result_df[correct == True]) / len(test_result_df)
        # metric_dict[f"certified_accuracy_{sigma}"][j] = certified_accuracy
        # metric_dict[sigma][j] = round(np.mean(radius), 3)
        metric_dict[sigma][j] = round(np.sum(test_result_df[correct == True][f'certified_radius_{sigma}']) / len(test_result_df), 3)
        # if sigma == augment_noise_std:
        #     certified_radius_dict[sigma][cur_key] = round(`np.mean(radius), 3)
                
    acc_df = pd.DataFrame(metric_dict)
    return acc_df, key_list, test_result_df_dict, noise_std_list

dir_lists = [
    # [
    #     # "mnist_dpsgd-auto-clip_cnn_sgd_lr_0.5_bsz_512_clip_bound_0.1_sigma_1.23Mar29_18-04-08",
    #     # "mnist_dpsgd-auto-clip_cnn_sgd_lr_0.5_bsz_512_clip_bound_0.1_sigma_1.23Mar29_18-04-21",
    #     "mnist_regular_cnn_sgd_lr_0.01_bsz_512_clip_bound_0.1_sigma_0.8Mar23_17-57-37",
    #     "mnist_dpsgd_cnn_sgd_lr_0.5_bsz_512_clip_bound_0.1_sigma_1.23Mar23_17-57-54",
    #     "mnist_dpsgd-auto-clip_cnn_sgd_lr_0.5_bsz_512_clip_bound_0.1_sigma_1.23_psac_TrueMay02_22-45-57",
    #     # "mnist_dpsgd-adv_cnn_sgd_lr_0.5_bsz_512_clip_bound_0.1_sigma_1.23_pgd_steps_20Apr11_23-03-29"
    # ],
    # [
    #     # "fashion-mnist_dpsgd-auto-clip_cnn_sgd_lr_4_bsz_2048_clip_bound_0.1_sigma_2.15Mar30_22-21-57",
    #     "fashion-mnist_regular_cnn_sgd_lr_0.01_bsz_512_clip_bound_0.1_sigma_0.8Mar30_22-20-58",
    #     "fashion-mnist_dpsgd_cnn_sgd_lr_4_bsz_2048_clip_bound_0.1_sigma_2.15Mar30_22-21-12",
    #     "fashion-mnist_dpsgd-auto-clip_cnn_sgd_lr_4_bsz_2048_clip_bound_0.1_sigma_2.15_psac_TrueMay03_00-42-09",
    #     # "fashion-mnist_dpsgd-adv_cnn_sgd_lr_4_bsz_2048_clip_bound_0.1_sigma_2.15_pgd_steps_20Apr11_23-12-23/"
    # ],
    # [
    #     "cifar10_regular_crossvit_tiny_240.in1k_sgd_lr_0.001_bsz_512_clip_bound_0.1_sigma_0.8May05_21-09-07",
    #     "cifar10_dpsgd_crossvit_tiny_240.in1k_adam_lr_0.001_bsz_1024_clip_bound_0.1_sigma_1.09619140625May05_21-34-38",
    #     "cifar10_dpsgd-auto-clip_crossvit_tiny_240.in1k_adam_lr_0.001_bsz_1024_clip_bound_0.1_sigma_1.09619140625_psac_TrueMay05_21-59-55"
    # ]
    [
        "fashion_mnist_ck_one_round_gamma2.0"
    ]
]
for dir_list in dir_lists:
    test_result_df_dict = {}
    metric_dict = defaultdict(dict)
    key_list = []
    certified_radius_dict = defaultdict(dict)
    noise_std_list = []
    for j, dir in enumerate(dir_list):
        
        dir = os.path.join("/data1/lwx/dp-cert-ours", dir)
        with open(os.path.join(dir, "config.json"), "r") as f:
            cfg = json.load(f)
        
        method = cfg['method']
        if cfg['method'] == "dpsgd-auto-clip":
            method += f"_psac_{cfg['psac']}"
        
        cur_key = f"{cfg['dataset']}_{method}"
        key_list.append(cur_key)
        # noise_std_list.append(augment_noise_std)

        test_result_df = pd.read_json(os.path.join(dir, "test_results.json"))
        # print(test_result_df)
        test_result_df_dict[cur_key] = test_result_df
        num_correct_at_radius = len(test_result_df[test_result_df['correct'] == True])
        accuracy = num_correct_at_radius / len(test_result_df)
        metric_dict["dataset"][j] = cfg['dataset']
        metric_dict["method"][j] = method
        metric_dict["accuracy"][j] = accuracy * 100
        
        for sigma in [0.25, 0.5, 1.0]:
            radius = test_result_df[f'certified_radius_{sigma}']
            correct = test_result_df[f'certified_correct_{sigma}']
            pred = test_result_df[f'certified_pred_{sigma}']
            # certified_accuracy = len(test_result_df[correct == True]) / len(test_result_df)
            # print(sigma, certified_accuracy)
            # metric_dict[f"certified_acc_{sigma}"][j] = certified_accuracy
            metric_dict[sigma][j] = round(np.sum(test_result_df[correct == True][f'certified_radius_{sigma}']) / len(test_result_df), 3)

            # if sigma == augment_noise_std:
            #     certified_radius_dict[sigma][cur_key] = np.mean(radius)

    print(pd.DataFrame(metric_dict).to_string())
    sigma_results = read_certified_robustness_results(key_list, test_result_df_dict, step_size=0.25)
    # print(pd.DataFrame(certified_radius_dict).to_string())
    for key in key_list:
        for sigma in [0.25, 0.5, 1.0]:
            # if augment_noise_std == sigma:
            print(f"{key} sigma={sigma}")
            # print(pd.DataFrame.from_dict(sigma_results[sigma][key]))
            avg = pd.DataFrame.from_dict(sigma_results[sigma][key]).iloc[0].tolist()
            print(avg)

            
# get cifar config list
dir_list = glob.glob("/data2/paulwu/robust-dp-label-noise/runs/cifar10_dpsgd-augment_resnet9_sgd_lr_0.25_bsz_512_clip_bound_0.1_sigma_1.54Apr22*")
trade_list = []
macer_list = []
consistency_list = []
vanilla_list = []
psac_list = []
for j, dir in enumerate(dir_list):
    with open(os.path.join(dir, "config.json"), "r") as f:
        cfg = json.load(f)
    if cfg["trades"]:
        trade_list.append(dir)
    elif cfg["macer"]:
        macer_list.append(dir)
    elif cfg["consistency"]:
        consistency_list.append(dir)
    else:
        vanilla_list.append(dir)


         dataset            method  accuracy  0.25    0.5    1.0
0  fashion-mnist  dpsgd-adv-smooth     84.52  0.56  0.783  0.556
fashion-mnist_dpsgd-adv-smooth sigma=0.25
[83.0, 76.7, 67.7, 54.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
fashion-mnist_dpsgd-adv-smooth sigma=0.5
[78.1, 70.7, 62.0, 52.3, 42.3, 31.3, 18.7, 0.0, 0.0, 0.0]
fashion-mnist_dpsgd-adv-smooth sigma=1.0
[44.6, 38.7, 33.5, 29.0, 23.9, 19.0, 15.7, 12.5, 10.1, 8.0]
